In [3]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=46e792f03ef18a23db12e78f8d53b403cd599cd0e0d3919a7c113f00d57e3aa8
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [6]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import wget
import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split


In [7]:
!wget -N "https://cainvas-static.s3.amazonaws.com/media/user_data/AmrutaKoshe/fruits.zip"
!unzip fruits.zip

Streaming output truncated to the last 5000 lines.
  inflating: fruits/train/train/Pepper Green/Pepper Green_209.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_21.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_210.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_211.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_212.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_213.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_214.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_215.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_216.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_217.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_218.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_219.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_22.jpg  
  inflating: fruits/train/train/Pepper Green/Pepper Green_220.jpg  

In [20]:
directory = '/content/fruits/train/train'
Name=[]
for file in os.listdir(directory):
    Name+=[file]

fruit_map = dict(zip(Name, [t for t in range(len(Name))]))
print(fruit_map)
r_fruit_map=dict(zip([t for t in range(len(Name))],Name))
print(r_fruit_map)

{'Raspberry': 0, 'Blueberry': 1, 'Tomato': 2, 'Clementine': 3, 'Apple Granny Smith': 4, 'Strawberry': 5, 'Watermelon': 6, 'Pomegranate': 7, 'Potato Red': 8, 'Papaya': 9, 'Plum': 10, 'Cherry': 11, 'Pear': 12, 'Pineapple': 13, 'Passion Fruit': 14, 'Mango': 15, 'Cucumber Ripe': 16, 'Cactus fruit': 17, 'Grape Blue': 18, 'Peach': 19, 'Pepper Green': 20, 'Apple Braeburn': 21, 'Avocado': 22, 'Limes': 23, 'Banana': 24, 'Cantaloupe': 25, 'Lemon': 26, 'Onion White': 27, 'Pepper Red': 28, 'Orange': 29, 'Corn': 30, 'Apricot': 31, 'Kiwi': 32}
{0: 'Raspberry', 1: 'Blueberry', 2: 'Tomato', 3: 'Clementine', 4: 'Apple Granny Smith', 5: 'Strawberry', 6: 'Watermelon', 7: 'Pomegranate', 8: 'Potato Red', 9: 'Papaya', 10: 'Plum', 11: 'Cherry', 12: 'Pear', 13: 'Pineapple', 14: 'Passion Fruit', 15: 'Mango', 16: 'Cucumber Ripe', 17: 'Cactus fruit', 18: 'Grape Blue', 19: 'Peach', 20: 'Pepper Green', 21: 'Apple Braeburn', 22: 'Avocado', 23: 'Limes', 24: 'Banana', 25: 'Cantaloupe', 26: 'Lemon', 27: 'Onion White',

In [16]:
print(len(Name))


3


In [21]:
img_datagen = ImageDataGenerator(rescale=1./255,
                                vertical_flip=True,
                                horizontal_flip=True,
                                rotation_range=40,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                zoom_range=0.1,
                                validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)


In [22]:
train_generator = img_datagen.flow_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=32,
                                                 subset='training',
                                                 target_size=(100, 100))

valid_generator = img_datagen.flow_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=16,
                                                 subset='validation',
                                                 target_size=(100, 100))

Found 13309 images belonging to 33 classes.
Found 3314 images belonging to 33 classes.


In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(100,100,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding = 'same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(len(fruit_map)))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 100, 100, 32)      896       
                                                                 
 max_pooling2d_36 (MaxPooli  (None, 50, 50, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_37 (Conv2D)          (None, 50, 50, 64)        18496     
                                                                 
 max_pooling2d_37 (MaxPooli  (None, 25, 25, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_38 (Conv2D)          (None, 25, 25, 64)        36928     
                                                                 
 max_pooling2d_38 (MaxPooli  (None, 12, 12, 64)       